In [1]:
import glob
import os
#import zipfile
import pandas as pd

df = pd.DataFrame(columns=['label', 'tiff_filename', 'index_r', 'index_g', 'index_b'])
# zipデータセット形式(zipファイル内にチャネルに別れたpngファイルがある。32x32) 10ファイルごとに1zipファイルを作成する
labels = [os.path.basename(x) for x in glob.glob("./data-org/*")]
labels

label_dict = {}
for i in range(len(labels)):
    label_dict[labels[i]] = i
label_dict

{'marker_black': 0,
 'ruler_black': 1,
 'pen_red': 2,
 'pencil_core': 3,
 'pen_green': 4,
 'paste': 5,
 'pen_multiclolor': 6,
 'marker_red': 7}

In [2]:
from PIL import Image
import numpy as np

i = 0
dst_dir = './data-tiff'
os.makedirs(dst_dir, exist_ok=True)
files = []
files_r = []
files_g = []
files_b = []

for file_path in glob.glob("./data-org/**/*.JPG", recursive=True):
    zip_filename = f'{i}.tiff'
    files.append(zip_filename)
    label = os.path.split(os.path.split(file_path)[0])[1]
    file_name = os.path.basename(file_path)
    base = os.path.splitext(file_name)[0]

    im = np.array(Image.open(file_path))
    im_R = im.copy()
    im_R[:, :, 1] = im_R[:, :, 0]
    im_R[:, :, 2] = im_R[:, :, 0]
    dst_name_r = f'{base}-R.png'
    files_r.append(dst_name_r)
    Image.fromarray(im_R).resize((128,128)).convert('L').save(os.path.join(dst_dir, dst_name_r))

    im_G = im.copy()
    im_G[:, :, 0] = im_G[:, :, 1]
    im_G[:, :, 2] = im_G[:, :, 1]
    dst_name_g = f'{base}-G.png'
    files_g.append(dst_name_g)
    Image.fromarray(im_G).resize((128,128)).convert('L').save(os.path.join(dst_dir, dst_name_g))

    im_B = im.copy()
    im_B[:, :, 0] = im_B[:, :, 2]
    im_B[:, :, 1] = im_B[:, :, 2]
    dst_name_b = f'{base}-B.png'
    files_b.append(dst_name_b)
    Image.fromarray(im_B).resize((128,128)).convert('L').save(os.path.join(dst_dir, dst_name_b))
    if len(files) > 100:
#        with zipfile.ZipFile(os.path.join(dst_dir, zip_filename), 'w', zipfile.ZIP_DEFLATED) as zip_file:
#            for j in range(len(files)):
#                zip_file.write(os.path.join(dst_dir, files_r[j]), files_r[j])
#                zip_file.write(os.path.join(dst_dir, files_g[j]), files_g[j])
#                zip_file.write(os.path.join(dst_dir, files_b[j]), files_b[j])
        stack = []
        for j in range(len(files)):
            stack.append(Image.open(os.path.join(dst_dir, files_r[j])).convert('L'))
            stack.append(Image.open(os.path.join(dst_dir, files_g[j])).convert('L'))
            stack.append(Image.open(os.path.join(dst_dir, files_b[j])).convert('L'))
            df = df.append({'label': label_dict[label], 'tiff_filename': os.path.join(dst_dir, zip_filename), 
                            'index_r': j*3, 'index_g': j*3+1, 'index_b': j*3+2}, ignore_index=True)
        stack[0].save(os.path.join(dst_dir, zip_filename), compression="tiff_deflate", save_all=True, append_images=stack[1:])
        files = []
        files_r = []
        files_g = []
        files_b = []
        i += 1

#with zipfile.ZipFile(os.path.join(dst_dir, zip_filename), 'w', zipfile.ZIP_DEFLATED) as zip_file:
#    for j in range(len(files)):
#        zip_file.write(os.path.join(dst_dir, files_r[j]), files_r[j])
#        zip_file.write(os.path.join(dst_dir, files_g[j]), files_g[j])
#        zip_file.write(os.path.join(dst_dir, files_b[j]), files_b[j])
stack = []
for j in range(len(files)):
    stack.append(Image.open(os.path.join(dst_dir, files_r[j])).convert('L'))
    stack.append(Image.open(os.path.join(dst_dir, files_g[j])).convert('L'))
    stack.append(Image.open(os.path.join(dst_dir, files_b[j])).convert('L'))
    df = df.append({'label': label_dict[label], 'tiff_filename': os.path.join(dst_dir, zip_filename), 
                    'index_r': j*3, 'index_g': j*3+1, 'index_b': j*3+2}, ignore_index=True)
stack[0].save(os.path.join(dst_dir, zip_filename), compression="tiff_deflate", save_all=True, append_images=stack[1:])

df.to_csv(os.path.join(dst_dir,'dataset-tiff.csv'))
df

,label,tiff_filename,index_r,index_g,index_b
0,1,./data-tiff/0.tiff,0,1,2
1,1,./data-tiff/0.tiff,3,4,5
2,1,./data-tiff/0.tiff,6,7,8
3,1,./data-tiff/0.tiff,9,10,11
4,1,./data-tiff/0.tiff,12,13,14
...,...,...,...,...,...
600,7,./data-tiff/5.tiff,285,286,287
601,7,./data-tiff/5.tiff,288,289,290
602,7,./data-tiff/5.tiff,291,292,293
603,7,./data-tiff/5.tiff,294,295,296
